**Transits:**

Som i nu ved kan vi (når geometrien tillader det) måle størrelsen på en exoplanet ved hjælp af transit metoden. Lidt mere specifikt måler vi transitdybden - altså størrelsesforholdet mellem planeten og stjernen ($\delta = R_{p}^2 / R_{*}^{2}$). Dette betyder også at præcisionen på vores måling af en planets størrelse afhænger både af hvor præcist vi kan måle størrelsesforholdet mellem planet og stjerne, samt hvor præcist vi kan bestemme stjernens absolutte størrelse.

**Opgave 1)** Hvad en henholdsvist transitdybden for Jorden og Jupiter omkring Solen?

Geometrien skal jo være rigtig for at vi kan se en planet med transit metoden.

**Opgave 2)** Hvis en anden (tilfældigt orienteret) civilisation kiggede på vores stjerne, Solen, hvad er så sandsynligheden for at de ville kunne se Jorden transitte? Hvad med Jupiter? Og hvad ville sandsynligheden være for at de kunne se en planet der kredsede 0.05 AU fra Solen (AU = Astronomisk Enhed = Afstanden mellem Jorden og Solen = 150 millioner km)?

Hvis i har mod på det kan i selv prøve at udlede formlen for at bestemme transit sandsynligheden. Alternativt kan i finde formlen (formel 8) i denne artikel: http://www.barnesos.net/publications/papers/2007.09.PASP.Barnes.eccentric.transiting.planets.pdf .


Lad os nu prøve at visualisere hvordan et transit ville se ud for henholdsvist Jorden og Jupiter hvis vi beholdte deres størrelser og rykkede dem ind i en afstand af ~0.05 AU af Solen.


For at gøre dette vil vi gøre brug af lidt Python kode. Har i ikke erfaring med python eller kodning i det hele taget? Frygt ej! Alt koden er nemlig skrevet på forhånd og alt i behøver at gøre er at køre hver kodeblok herunder (kan gøres ved at markere en celle og bruge ctrl+enter). Koden vi bruger her er ret simpel, så prøv at følg med i hvad der sker undervejs. Alt tekst der står efter et '#' på en linje er bare kommentarer og burde gøre det lettere at følge med i hvad der sker.

En af fordelene ved python er brugen af pakker. Altså kode som andre har skrevet og som vi kan gøre brug af og dermed slippe for at kode alt op fra bunden. Så lad os starte med at (installere og) importere de pakker vi får brug for:

In [ ]:
import os

package = "batman-package"
try:
    __import__package
except:
    os.system("pip install "+ package)    

import numpy as np
import batman
import matplotlib.pyplot as plt

numpy er et bibliotek der tilføjer understøttelse til store multidimensionelle arrays og matrixer sammen med en stor samling matematiske funktioner til at betjene disse arrays.

batman pakken (Bad-Ass Transit Model cAlculatioN) vi lige har installeret indeholder en funktion der kan hjælpe os med at beregne en lyskurve givet et sæt af inputparametre. I praksis er det en python implementering af den analytiske lyskurve funktion fra denne artikel: https://arxiv.org/pdf/astro-ph/0210099.pdf

matplotlib bruger vi til at kunne plotte vores resultater.

In [ ]:
#Vi starter først med at definere vores funktion:
def transitfunction(t, rp): #Funktionen vil have et tidsarray (t) og en planetstørrelse (rp) som input. Resten af parametrene er allerede sat inde i funktionen
  params = batman.TransitParams()
  params.t0 = 0.                       #time of inferior conjunction
  params.per = 1.75                    #orbital period
  params.rp = rp                       #planet radius (in units of stellar radii)
  params.a = 11.                       #semi-major axis (in units of stellar radii)
  params.inc = 87.                     #orbital inclination (in degrees)
  params.ecc = 0.                      #eccentricity
  params.w = 90.                       #longitude of periastron (in degrees)
  params.u = [0.1, 0.3]                #limb darkening coefficients [u1, u2]
  params.limb_dark = "quadratic"       #limb darkening model

  m = batman.TransitModel(params, t)    #initializes model
  flux = m.light_curve(params)          #calculates light curve
  return flux

#Herefter har vi lige brug for et par størrelser som vi skal brug til at omregne til andre enheder
jupiter_radius_km = 69950  #Jupiters radius i kilometer
earth_radius_km   = 6371   #Jordens radius i kilometer
sun_radius_km     = 696700 #Solens radius i kilometer

#Da vores transit funktion bruger radius af planeten givet i enheder af stjernes radius omregner vi lige:
jupiter_sun_radius_ratio = jupiter_radius_km / sun_radius_km
earth_sun_radius_ratio = earth_radius_km / sun_radius_km

print("R_Jupiter / R_sun =", round(jupiter_sun_radius_ratio,4))
print("R_earth / R_sun =", round(earth_sun_radius_ratio,4))

t = np.linspace(-0.025, 0.025, 300) #Her laver vi et array som indeholder 300 værdier jævnt fordelt mellem intervallet -0.025 til 0.025. Det bruger vi til at bestemme til hvilke 'tidspunkter' vi gerne vil beregne fluxen i vores model.

flux_jupiter = transitfunction(t=t, rp=jupiter_sun_radius_ratio) #Her kalder vi vores funktion med ovenstående array indeholdende tidspunkter samt Jupiters radius målt i Solradiier som input. Vi gemmer de beregnede output fluxes i flux_jupiter
flux_earth = transitfunction(t=t, rp=earth_sun_radius_ratio)     #Samme som ovenstående linje, men med Jordens radius som input

#Og så plotter vi vores resultater:
plt.rcParams['figure.figsize'] = 15, 10 #Sætter størrelsen på plottet. I kan ændre de to tal efter behov
plt.plot(t, flux_jupiter, label="Jupiter")
plt.plot(t, flux_earth, label="Earth")
plt.xlabel("Time from central transit")
plt.ylabel("Relative flux")
plt.legend()
plt.show()


Kombineret skulle forskellen i transit sandsynlighed mellem planeter tæt og langt fra deres stjerne og ovenstående plot gerne give en ide om hvorfor det er betydeligt lettere at finde store planeter tæt på deres stjerner.

I ovenstående eksempel kender vi jo allerede størrelsen på Jorden og Jupiter, men i praksis er dette jo netop hvad vi forsøger at bestemme.

Så lad os nu antage at vi har gjort os en række målinger af en stjernes lys til nogle bestemte tidspunkter. Dem gemmer vi i to arrays:

In [ ]:
data_time_array = [-0.05,       -0.04655172, -0.04310345, -0.03965517, -0.0362069,  -0.03275862,
 -0.02931034, -0.02586207, -0.02241379, -0.01896552, -0.01551724, -0.01206897,
 -0.00862069, -0.00517241, -0.00172414,  0.00172414,  0.00517241,  0.00862069,
  0.01206897,  0.01551724,  0.01896552,  0.02241379, 0.02586207,  0.02931034,
  0.03275862,  0.0362069,   0.03965517,  0.04310345,  0.04655172,  0.05]

data_flux_array = [1,         1,         1,         1,         0.99980163, 0.99567555,
 0.98936876, 0.98256495, 0.97661546, 0.9736041,  0.97276448, 0.97225026,
 0.97192661, 0.97173464, 0.97164497, 0.97164497, 0.97173464, 0.97192661,
 0.97225026, 0.97276448, 0.9736041,  0.97661546, 0.98256495, 0.98936876,
 0.99567555, 0.99980163, 1,         1,         1,         1,        ]


Lad os starte med at plotte vores data:

In [ ]:
plt.rcParams['figure.figsize'] = 15, 10
plt.plot(data_time_array, data_flux_array,'r.', label="Data")
plt.xlabel("Time from central transit")
plt.ylabel("Relative flux")
plt.ylim(0.97,1.002)
plt.legend()
plt.show()

Det næste skridt er nu at vi vil prøve at bestemme nogle af planetens parametre. Som i har set i vores transit funktion højere oppe er der en del parametre der indgår, men for nemheds skyld er de fleste af parametrene allerede sat til deres sande værdi i nedenstående. Vi mangler dog at finde planetens radius og den halve storakse (i dette tilfælde er excentriciteten = 0 og dermed har vi egentligt bare en cirkulær bane hvor vi prøver at bestemme radius).

**Opgave 3)** Varier parametrene 'Rp_guess' og 'a_guess' i nedenstående indtil jeres model passer bedst muligt med dataen. I kan se hvor godt jeres 'fit' passer visuelt i figuren, men vi kan også sætte et tal på det. Der er mange måder at gøre det, men da vi her har data helt uden støj på kan vi nøjes med at kigge på den gennemsnitlige absolutte fejl. Det er simpelt bare et mål for den gennemsnitlige fejl mellem parrede observationer, hvilket betyder at des mindre dette tal er, des bedre er vores fit.

In [ ]:
def transitfunction(t, a, rp):
  params = batman.TransitParams()
  params.t0 = 0.                        #time of inferior conjunction
  params.per = 1.75                     #orbital period
  params.rp = rp                        #planet radius (in units of stellar radii)
  params.a = a                          #semi-major axis (in units of stellar radii)
  params.inc = 85.                      #orbital inclination (in degrees)
  params.ecc = 0.                       #eccentricity
  params.w = 90.                        #longitude of periastron (in degrees)
  params.u = [0.1, 0.3]                 #limb darkening coefficients [u1, u2]
  params.limb_dark = "quadratic"        #limb darkening model

  m = batman.TransitModel(params, t)    #initializes model
  flux = m.light_curve(params)          #calculates light curve
  return flux

t_long = np.linspace(-0.05, 0.05, 300) #Vi har et tidsarray med 300 værdier, simpelt for at plottet bliver mere 'smooth' at se på.
t  = np.linspace(-0.05, 0.05, 30)      #Og så har vi et tidsarray med 30 værdier (samme længde og tidspunkter som vores datasæt) som vi kan bruge til at beregne den gennemsnitlige absolutte fejl.

a_guess = 8 #<-- Ændr dette input
rp_guess = 0.15 #<-- Og også dette

fit_for_plot    = transitfunction(t=t_long, a=a_guess, rp=rp_guess)
fit_for_MAEcalc = transitfunction(t=t,a=a_guess, rp=rp_guess)

mae = np.mean(np.abs(data_flux_array-fit_for_MAEcalc)) #Her beregner vi den gennemsnitlige absolutte fejl mellem dataen og vores model
print("MEAN ABSOLUTE ERROR:", mae)

plt.plot(t_long, fit_for_plot, label="Fit")
plt.plot(data_time_array, data_flux_array,'r.', label="Data")
plt.xlabel("Time from central transit")
plt.ylabel("Relative flux")
plt.legend()
plt.show()

Her har vi kigget på et syntetisk datasæt uden nogle systematisk fejl eller nogle former for støj. Sådan ser det selvfølgelig aldrig ud i den virkelige verden, men illustrerer fint hvordan man i praksis kan bestemme nogle af de fysiske egenskaber ved en planet. Normalt fitter vi heller ikke  'i hånden' som i lige har gjort, men gør i stedet brug af divserse sampling teknikker til at bestemme alle de mange parametre og deres usikkerheder på samme tid, mens vi tager højde for usikkerhederne på vores input data.

Vi kan bestemme varigheden af et transit ved:

$T_{duration} \approx \frac{P \cdot R_{*}}{\pi \cdot a}$,

hvor P = perioden, $R_{*}$ = stjernens radius og a = den halve storakse. 

**Opgave 4)** Antag P = 1.75 dage, $R_{*}$ = 0.79 x solens radius og a = den halve storakse i bestemte i ovenstående. Hvad er da denne planets transit varighed?

**Opgave 5)** Gå til https://catalogs.mast.stsci.edu/eaot og se om i fra transit varigheden og planetens størrelse kan gætte hvilken planet der er brugt til at producere det data vi har kigget på i ovenstående. HINT: Når i søger i kataloget kan det være en ide at sætte en 'range' for de parametre i sætter ind. Vær også opmærksom på, at planet radius er givet i Jupiter radiier og at transit varigheden er givet i timer.

